In [1]:
pip install opencv-python numpy 


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np

# Video file in the same folder
cap = cv2.VideoCapture("video.mp4")

if not cap.isOpened():
    print("Error: Video not found")
    exit()

bg_subtractor = cv2.createBackgroundSubtractorMOG2(
    history=100,
    varThreshold=40
)

heatmap = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))

    fg_mask = bg_subtractor.apply(frame)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

    if heatmap is None:
        heatmap = np.zeros_like(fg_mask, dtype=np.float32)

    heatmap += fg_mask.astype(np.float32)

    heatmap_norm = cv2.normalize(
        heatmap, None, 0, 255, cv2.NORM_MINMAX
    ).astype(np.uint8)

    heatmap_color = cv2.applyColorMap(
        heatmap_norm, cv2.COLORMAP_JET
    )

    contours, _ = cv2.findContours(
        fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    for cnt in contours:
        if cv2.contourArea(cnt) > 800:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(
                frame, (x, y),
                (x + w, y + h),
                (0, 255, 0), 2
            )

    output = cv2.addWeighted(frame, 0.7, heatmap_color, 0.3, 0)

    cv2.imshow("Motion Detection Heatmap", output)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
